# **Aspect Based Customer Review Classification & Summarization**

## **Business Context**

In today's digital era, **customer reviews are pivotal for e-commerce platforms and online service providers**, serving as a barometer for customer satisfaction, highlighting areas for improvement, and guiding business decisions. The intricate process of Aspect-based Sentiment Analysis is instrumental in dissecting these reviews, offering a granular understanding of customer sentiments towards various facets of a product or service. This nuanced approach enables businesses to pinpoint specific elements—be it the screen, keyboard, or customer service of a laptop—that may need enhancement.

Moreover, the advent of Large Language Models (LLMs) has revolutionized the way businesses approach the summarization of customer reviews and case briefs. By leveraging LLMs, companies can swiftly and accurately distill the essence of customer feedback, streamlining the process of understanding overall sentiments and facilitating the refinement of product offerings.

## **Project Objective**

Developing a Generative AI application using a Large Language Model to **automate the Asspect-based Classification and Summarization of Customer Reviews** received by a business. The application will aim to predict review sentiment, aspect-based sentiment and generate summaries of these customer reviews, and will evaluate the performance of the LLM at these tasks through various evaluation schemes.



This project involves two distinct sections:

1. Aspect-Based Sentiment Classification
2. Parameter-Efficient Fine-Tuning and Evaluation

## **Part 1: Aspect Based Sentiment Classification**

In [ ]:
# Installation for GPU llama-cpp-python==0.2.28
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --upgrade --no-cache-dir --verbose -q
# Installation for the huggingface-hub==0.23.4
!pip install huggingface_hub==0.23.4 -q
# installation for the datasets==2.16.1
!pip install datasets==2.16.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 83.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 142.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 38.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.28-cp310-cp310-linux_x86_64.whl size=8778632 sha256=c7aa65807e8cba84dbd111265e50366bc5e0db4583386ca178e200c768df633f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5gni8iro/wheels/93/6e/a9/478cce089dc2a082bdcffe468a1c65465c91b25d911b30da82
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successf

In [ ]:
# Installing numpy version 1.25.2
!pip install numpy==1.25.2

**1.1 (B) Import necessary libraries**

In [ ]:
#  Import hf_hub_download from the huggingface_hub
from huggingface_hub import hf_hub_download

#  Import Llama from the llama_cpp library
from llama_cpp import Llama

# Import load_dataset from datasets
from datasets import load_dataset

In [ ]:
from sklearn.model_selection import train_test_split # Load train_test_split from sklean library
from sklearn.metrics import f1_score

from google.colab import drive
import datasets
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

### **1.2 Model Setup**

Setup model name and base name

Create model path variable

In [ ]:
# Setting up model from Huggine Face
#  model name should be the "Llama-13B-chat-GGUF" from "TheBloke"
model_name = "TheBloke/Llama-2-13B-chat-GGUF"


# model basename should be the Quantized model we want to use here - This should be the "Q5_K_M.gguf", the model is in gguf format
model_basename = "llama-2-13b-chat.Q5_K_M.gguf"


In [ ]:
# Downloading model from the Hugging Face Hub
model_path = hf_hub_download(
    repo_id= model_name, # mention the variable "model_name" here
    filename= model_basename  # mention the variable "model_basename" here
    )

In [ ]:
# Assigning the Llama class to the variable lcpp_llm
lcpp_llm = Llama(
        model_path=model_path,
        n_threads=2,  # CPU cores
        n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
        n_ctx=4096,  # Context window
    )

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


### **1.3 Data Preparation**

(A) Upload and read csv

(B) Product Based DataFrame creation



**1.3 (A) Upload and read csv**

In [ ]:
# Read dataset using pandas library
import pandas as pd
sample_reviews_df = pd.read_csv("/content/customer_reviews_dataset.csv")

In [ ]:
# Checking the first 5 sample of the dataset
sample_reviews_df.head()

,customer_id,customer_name,product_name,product_type,review_date,rating,review_sentiment,review_text,aspects_review,response,summary
0,CID041,Aisha Patel,Dell Inspiron 15,Laptop,01/01/2024,5,Positive,I bought this laptop for my son who is studyin...,"{ battery : Positive , keyboard : Positive, di...",It's fantastic to hear that the laptop you pur...,"The user purchased a laptop for their son, who..."
1,CID011,Liam Thomson,JBL Tune 500BT,Headphones,01/05/2024,1,Negative,I was very disappointed with these headphones....,"{ sound : Negative , comfort : Negative , char...",I'm truly sorry to hear about your disappointi...,The user expressed disappointment with poor so...
2,CID034,Maria García,Mi Power Bank 3i,Power Bank,01/10/2023,4,Positive,"Awesome power bank, it charges my phone very f...","{ battery : Positive , charging : Positive }",Thank you for your positive review of our powe...,The user praises the power bank for its fast c...
3,CID032,Jamal Johnson,Samsung Galaxy S21,Smartphone,03/03/2023,2,Negative,I bought this phone mainly for its much-hyped ...,"{ display : Positive , camera : Negative , Bat...",I'm truly sorry to hear that the camera perfor...,The user expressed disappointment with the pho...
4,CID051,Emily Nguyen,JBL Tune 500BT,Headphones,05/25/2023,5,Positive,"I love these headphones, they are amazing. The...","{ sound : Positive , comfort : Positive , char...",Thank you for your wonderful feedback on our h...,The user praises the headphones for their exce...


**1.3 (B) Product Based DataFrame creation**

In [ ]:
# Creating a "Laptop" reviews DataFrame based on the "product_type" column in the dataset
laptop_reviews = sample_reviews_df[sample_reviews_df['product_type'] == 'Laptop']

# Creating a "Laptop" reviews DataFrame based on the "product_type" column in the dataset for only positive reviews
# positive_laptop_reviews_df = laptop_reviews[laptop_reviews['review_sentiment'] == 'Positive']

# Creating a "Headphones" reviews DaatFrame based on the "product_type" column in the dataset
headphones_reviews = sample_reviews_df[sample_reviews_df['product_type'] == 'Headphones']

# Creating a "Smartphone" reviews DaatFrame based on the "product_type" column in the dataset
smartphone_reviews = sample_reviews_df[sample_reviews_df['product_type'] == 'Smartphone']

# Creating a "Power Bank" reviews DaatFrame based on the "product_type" column in the dataset
power_bank_reviews = sample_reviews_df[sample_reviews_df['product_type'] == 'Power Bank']

In [ ]:
# Selecting gold examples as test data
import json
laptop_gold_examples = laptop_reviews.sample(2, random_state=40)
headphones_gold_examples = headphones_reviews.sample(2, random_state=40)
smartphone_gold_examples = smartphone_reviews.sample(2, random_state=40)
power_bank_gold_examples = power_bank_reviews.sample(2, random_state=40)

# Concatenate positive and negative gold examples
sample_reviews_gold_examples_df = pd.concat([laptop_gold_examples, headphones_gold_examples,smartphone_gold_examples,power_bank_gold_examples])

# Creating the training set by excluding gold examples
sample_reviews_examples_df = sample_reviews_df.drop(index=sample_reviews_gold_examples_df.index)

# Converting gold examples to JSON
columns_to_select = ['review_text', 'product_type','aspects_review']
gold_examples_json = sample_reviews_gold_examples_df[columns_to_select].to_json(orient='records')

# Print the first record from the JSON
print(json.loads(gold_examples_json)[0])

# Print the shapes of the datasets
print("Training Set Shape:", sample_reviews_examples_df.shape)
print("Gold Examples Shape:", sample_reviews_gold_examples_df.shape)


{'review_text': "Originally bought it for my work, quite happy with it so far! Fast, reliable, easy to use and has a good webcam. Display is good and battery backup is also great. The keyboard is a joy to type on, gives me the old typewriter vibes! Quickly become my main laptop for everyday use, and I'm very satisfied with my purchase.", 'product_type': 'Laptop', 'aspects_review': '{ battery : Positive , keyboard : Positive , display : Positive }'}
Training Set Shape: (22, 11)
Gold Examples Shape: (8, 11)


The training set has 22 instances and 11 columns(Features).  22 rows and 11 columns.  While the test set(gold examples) has 8 instances and 11 features(columns).  

In [ ]:
# Checking the first 3 rows of the test data
sample_reviews_gold_examples_df.head(3)

,customer_id,customer_name,product_name,product_type,review_date,rating,review_sentiment,review_text,aspects_review,response,summary
27,CID053,Pooja Jain,Dell Inspiron 15,Laptop,12/31/2023,4,Positive,"Originally bought it for my work, quite happy ...","{ battery : Positive , keyboard : Positive , d...",Thank you for sharing your positive experience...,The customer is satisfied with their work lapt...
14,CID004,Bradley Wiggins,Dell Inspiron 15,Laptop,05/10/2024,5,Positive,"This is a great laptop, I am very happy with i...","{ battery : Positive , keyboard : Positive , d...",Thank you for your positive feedback on our la...,"The customer is satisfied with their laptop, s..."
23,CID075,Jack Ryder,JBL Tune 500BT,Headphones,05/05/2023,5,Positive,Great for listening to music or podcasts. Good...,"{ sound : Positive , comfort : Positive , char...",Thank you for your positive review of our head...,Customer praises headphones for good sound qua...


In [ ]:
#  selecting 8 samples from the gold examples and converting it to json
gold_examples = (
        sample_reviews_gold_examples_df.loc[:, columns_to_select]
                                     .sample(8, random_state=40) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

### **1.4 Derive Prompt**

Create a Few Shot System Message

Combine user_prompt and system_message to create the prompt

The functions that we used to assemble examples, create prompts and to evaluate these prompts are broadly the same as in the case of sentiment analysis. However, we should adapt the few-shot examples to now account for the multiple aspects.

**Create a Few Shot System Message**

In [ ]:
# Creating few shot system message accounting for the mulitple project aspects
few_shot_system_message = """
<<SYS>>Perform aspect based sentiment analysis on customer reviews presented in the input delimited by triple backticks, that is, ```.

For each review presented as input:
- You need to identify firstly the product_type based on the review . Identify only 1 out of the 4 product_types : Laptop , Headphones , Power Bank , Smartphone
- After identifying the product_type , you will need to assign a sentiment polarity for each aspect of the product , it should only be 'Positive' or 'Negative' , nothing else is acceptable .
- Based on the product_type identified , give output only in 1 of the below 4 format :
1. if Laptop identified then - [ Laptop : { battery : ... , keyboard : ... , display : ... } ]
2. if Headphone identified then - [ Headphone : { sound : ... , comfort : ...  , charging : ... } ]
3. if Power Bank identified then - [ Power Bank : { battery : ...  , charging : ...  } ]
4. if Smartphone identified then - [ Smartphone : { display : ... , camera : ... , Battery : ... } ]
- Here '...' represent the sentiment polarity ( Positive or Negative ).
- Aspects mentioned for each product type are fixed and no aspect should be added or deleted
- Stricltly give output in above format. If any other format , then regenerate the output to the above mentioned format
- Do not explain your answer

following are the user and system conversation examples

<</SYS>>
"""

**Combine user_prompt and system_message to create the prompt**

In [ ]:
def generate_llama_response( system_message ,  few_shot_examples  , new_review , temp ):

    # Combine user_prompt and system_message to create the prompt
    prompt = f"[INST]{system_message}\n{few_shot_examples}\n{'user'}: ```{user_message_template.format(review=new_review)}```[INST]"
    print(prompt) # To see what the model is actually seeing


    # Generate a response from the LLM model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=256,
        temperature=temp,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]
    return response_text



In [ ]:
def create_examples_with_seed(dataset, n=2, random_seed=None):
    """
    Return two DataFrames with randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (text + label)
        n (int): number of examples of each class to be selected
        random_seed (int): seed for reproducibility (default is None)

    Output:
        few_shot_examples_df (DataFrame): A DataFrame with examples in random order
        new_df (DataFrame): A new DataFrame excluding selected examples
    """

    laptop_reviews = (dataset.product_type == 'Laptop')
    headphone_reviews = (dataset.product_type == 'Headphones')
    power_bank_reviews = (dataset.product_type == 'Power Bank')
    smartphone_reviews = (dataset.product_type == 'Smartphone')
    columns_to_select = ['review_text', 'product_type','aspects_review']

    # Set a fixed random seed for reproducibility
    np.random.seed(random_seed)

    laptop_examples = dataset.loc[laptop_reviews, columns_to_select].sample(n)
    headphone_examples = dataset.loc[headphone_reviews, columns_to_select].sample(n)
    power_bank_examples = dataset.loc[power_bank_reviews, columns_to_select].sample(n)
    smartphone_examples = dataset.loc[smartphone_reviews, columns_to_select].sample(n)

    few_shot_examples_df = pd.concat([laptop_examples, headphone_examples, power_bank_examples, smartphone_examples])

    # sampling without replacement is equivalent to random shuffling
    few_shot_examples_df = few_shot_examples_df.sample( 4*n, replace=False)

    # Create a new DataFrame excluding selected examples
    new_df = dataset.drop(index=few_shot_examples_df.index)

    return few_shot_examples_df, new_df


In [ ]:
def compute_combined_aspect_and_product_accuracy(df):
    correct_predictions_count = 0

    # Function to parse aspect-based sentiment string into a dictionary
    def parse_aspects(aspect_string):
        aspect_string= str(aspect_string)
        aspect_string = re.sub(r'[{}]', '', aspect_string)  # Remove curly braces
        aspects = re.split(r',\s*', aspect_string)  # Split into individual aspects

        try:
         dict_res = dict(re.split(r'\s*:\s*', aspect) for aspect in aspects if aspect)
        except:
          dict_res={}

        return dict_res


    # Function to normalize text by removing spaces and converting to lowercase
    def normalize_text(text):
        return ''.join(text.split()).lower() if text is not None else None

    # Iterate over each row to compare product type and aspect-based sentiments
    for index, row in df.iterrows():
        predicted_product_type = normalize_text(row['predicted_product_type'])
        actual_product_type = normalize_text(row['product_type'])
        predicted_aspects = parse_aspects(row['predicted_aspect_based_sentiment'])
        actual_aspects = parse_aspects(row['aspects_review'])

        # Check if product type matches and all aspects match in sentiment
        if predicted_product_type == actual_product_type and \
          predicted_aspects != '' and \
          all(predicted_aspects.get(key, '').lower() == value.lower() for key, value in actual_aspects.items()):
            correct_predictions_count += 1

    # Calculate accuracy
    accuracy = (correct_predictions_count / len(df)) * 100
    return accuracy

###**Measuring prompt performance**

Generate response and compute accuracy

Evaluate the results

In [ ]:
user_message_template = "{review}"

In [ ]:
def extract_product_type(text):
    text=str(text)
    match = re.search(r'\[([^:]+)', text)
    return match.group(1).strip() if match else None

def extract_aspect_based_sentiment(text):
    text=str(text)
    match = re.search(r'\{([^}]+)\}', text)
    return "{ " + match.group(1).strip() + " }" if match else None

In [ ]:
accuracy_list = []

In [ ]:
sample_reviews_examples_df.head()

,customer_id,customer_name,product_name,product_type,review_date,rating,review_sentiment,review_text,aspects_review,response,summary
0,CID041,Aisha Patel,Dell Inspiron 15,Laptop,01/01/2024,5,Positive,I bought this laptop for my son who is studyin...,"{ battery : Positive , keyboard : Positive, di...",It's fantastic to hear that the laptop you pur...,"The user purchased a laptop for their son, who..."
1,CID011,Liam Thomson,JBL Tune 500BT,Headphones,01/05/2024,1,Negative,I was very disappointed with these headphones....,"{ sound : Negative , comfort : Negative , char...",I'm truly sorry to hear about your disappointi...,The user expressed disappointment with poor so...
2,CID034,Maria García,Mi Power Bank 3i,Power Bank,01/10/2023,4,Positive,"Awesome power bank, it charges my phone very f...","{ battery : Positive , charging : Positive }",Thank you for your positive review of our powe...,The user praises the power bank for its fast c...
4,CID051,Emily Nguyen,JBL Tune 500BT,Headphones,05/25/2023,5,Positive,"I love these headphones, they are amazing. The...","{ sound : Positive , comfort : Positive , char...",Thank you for your wonderful feedback on our h...,The user praises the headphones for their exce...
6,CID043,Sofia Chen,Dell Inspiron 15,Laptop,05/20/2023,1,Negative,"I bought this laptop a month ago, and it's alr...","{ battery : Negative , keyboard : Negative , d...",I'm sorry to hear about your laptop issues and...,"The customer reports issues with their laptop,..."


**Generate response and compute accuracy**

In [ ]:
for i in range(2):
    few_shot_examples_df ,sample_reviews_df = create_examples_with_seed(sample_reviews_examples_df, n=1 , random_seed = i)

    review_example1 = few_shot_examples_df.iloc[0].review_text
    review_example2 = few_shot_examples_df.iloc[1].review_text
    review_example3 = few_shot_examples_df.iloc[2].review_text
    review_example4 = few_shot_examples_df.iloc[3].review_text

    assistant_output_example1 = "[ " + few_shot_examples_df.iloc[0].product_type.lower() + " : " + few_shot_examples_df.iloc[0].aspects_review.lower() + " ]"
    assistant_output_example2 = "[ " + few_shot_examples_df.iloc[1].product_type.lower() + " : " + few_shot_examples_df.iloc[1].aspects_review.lower() + " ]"
    assistant_output_example3 = "[ " + few_shot_examples_df.iloc[2].product_type.lower() + " : " + few_shot_examples_df.iloc[2].aspects_review.lower() + " ]"
    assistant_output_example4 = "[ " + few_shot_examples_df.iloc[3].product_type.lower() + " : " + few_shot_examples_df.iloc[3].aspects_review.lower() + " ]"

    few_shot_examples = [
        {'role':'user', 'content': user_message_template.format(review=review_example1)},
        {'role':'assistant', 'content': f"{assistant_output_example1}"},
        {'role':'user', 'content': user_message_template.format(review=review_example2)},
        {'role':'assistant', 'content': f"{assistant_output_example2}"},
        {'role':'user', 'content': user_message_template.format(review=review_example3)},
        {'role':'assistant', 'content': f"{assistant_output_example3}"},
        {'role':'user', 'content': user_message_template.format(review=review_example4)},
        {'role':'assistant', 'content': f"{assistant_output_example4}"}
                        ]

    few_shot_examples_str = json.dumps(few_shot_examples)


    sample_reviews = sample_reviews_df.review_text.values
    sentiment_predictions = []
    # generate_llama_response( few_shot_system_message , few_shot_examples_str  , input_text , 0.1 )
    for sample_review in tqdm(sample_reviews):
        try:
            sentiment_predictions.append(generate_llama_response( few_shot_system_message , few_shot_examples_str , sample_review , 0.1 ))
        except Exception as e:
            print(e)
            sentiment_predictions.append("")
    sentiment_predictions
    sample_reviews_df["sentiment_prediction"] = sentiment_predictions

    pattern = r'\[\s*.*?\s*:\s*\{.*?\}\s*\]'

    # Function to extract the sentiment data
    def extract_sentiment(text):
        match = re.findall(pattern, text)
        return match[0] if match else None


    # Apply the function to each row in the DataFrame
    sample_reviews_df['extracted_sentiment'] = sample_reviews_df['sentiment_prediction'].apply(extract_sentiment)

    sample_reviews_df
    # Function to extract product type
    print(sample_reviews_df['extracted_sentiment'])

    # Apply the functions to each row in the DataFrame
    try:
          sample_reviews_df['predicted_product_type'] = sample_reviews_df['extracted_sentiment'].apply(extract_product_type)
          sample_reviews_df['predicted_aspect_based_sentiment'] = sample_reviews_df['extracted_sentiment'].apply(extract_aspect_based_sentiment)
    except Exception as e:
            print(e)
            print(sample_reviews_df['predicted_product_type'] )

    sample_reviews_df.predicted_aspect_based_sentiment.value_counts()



    # Sample usage with your DataFrame
    combined_accuracy = compute_combined_aspect_and_product_accuracy(sample_reviews_df)

    print(f"Combined Product Type and Aspect-Based Sentiment Accuracy: {combined_accuracy}%")
    res = combined_accuracy
    accuracy_list.append(res)

  0%|          | 0/18 [00:00<?, ?it/s]

[INST]
<<SYS>>Perform aspect based sentiment analysis on customer reviews presented in the input delimited by triple backticks, that is, ```.

For each review presented as input:
- You need to identify firstly the product_type based on the review . Identify only 1 out of the 4 product_types : Laptop , Headphones , Power Bank , Smartphone
- After identifying the product_type , you will need to assign a sentiment polarity for each aspect of the product , it should only be 'Positive' or 'Negative' , nothing else is acceptable .
- Based on the product_type identified , give output only in 1 of the below 4 format :
1. if Laptop identified then - [ Laptop : { battery : ... , keyboard : ... , display : ... } ]
2. if Headphone identified then - [ Headphone : { sound : ... , comfort : ...  , charging : ... } ]
3. if Power Bank identified then - [ Power Bank : { battery : ...  , charging : ...  } ]
4. if Smartphone identified then - [ Smartphone : { display : ... , camera : ... , Battery : ... }

  6%|▌         | 1/18 [01:30<25:34, 90.28s/it]Llama.generate: prefix-match hit


[INST]
<<SYS>>Perform aspect based sentiment analysis on customer reviews presented in the input delimited by triple backticks, that is, ```.

For each review presented as input:
- You need to identify firstly the product_type based on the review . Identify only 1 out of the 4 product_types : Laptop , Headphones , Power Bank , Smartphone
- After identifying the product_type , you will need to assign a sentiment polarity for each aspect of the product , it should only be 'Positive' or 'Negative' , nothing else is acceptable .
- Based on the product_type identified , give output only in 1 of the below 4 format :
1. if Laptop identified then - [ Laptop : { battery : ... , keyboard : ... , display : ... } ]
2. if Headphone identified then - [ Headphone : { sound : ... , comfort : ...  , charging : ... } ]
3. if Power Bank identified then - [ Power Bank : { battery : ...  , charging : ...  } ]
4. if Smartphone identified then - [ Smartphone : { display : ... , camera : ... , Battery : ... }

 11%|█         | 2/18 [01:50<13:02, 48.91s/it]Llama.generate: prefix-match hit


...

[INST]
<<SYS>>Perform aspect based sentiment analysis on customer reviews presented in the input delimited by triple backticks, that is, ```.

For each review presented as input:
- You need to identify firstly the product_type based on the review . Identify only 1 out of the 4 product_types : Laptop , Headphones , Power Bank , Smartphone
- After identifying the product_type , you will need to assign a sentiment polarity for each aspect of the product , it should only be 'Positive' or 'Negative' , nothing else is acceptable .
- Based on the product_type identified , give output only in 1 of the below 4 format :
1. if Laptop identified then - [ Laptop : { battery : ... , keyboard : ... , display : ... } ]
2. if Headphone identified then - [ Headphone : { sound : ... , comfort : ...  , charging : ... } ]
3. if Power Bank identified then - [ Power Bank : { battery : ...  , charging : ...  } ]
4. if Smartphone identified then - [ Smartphone : { display : ... , camera : ... , Battery : ... }

 94%|█████████▍| 17/18 [15:46<01:04, 64.11s/it]Llama.generate: prefix-match hit


[INST]
<<SYS>>Perform aspect based sentiment analysis on customer reviews presented in the input delimited by triple backticks, that is, ```.

For each review presented as input:
- You need to identify firstly the product_type based on the review . Identify only 1 out of the 4 product_types : Laptop , Headphones , Power Bank , Smartphone
- After identifying the product_type , you will need to assign a sentiment polarity for each aspect of the product , it should only be 'Positive' or 'Negative' , nothing else is acceptable .
- Based on the product_type identified , give output only in 1 of the below 4 format :
1. if Laptop identified then - [ Laptop : { battery : ... , keyboard : ... , display : ... } ]
2. if Headphone identified then - [ Headphone : { sound : ... , comfort : ...  , charging : ... } ]
3. if Power Bank identified then - [ Power Bank : { battery : ...  , charging : ...  } ]
4. if Smartphone identified then - [ Smartphone : { display : ... , camera : ... , Battery : ... }

100%|██████████| 18/18 [16:09<00:00, 53.88s/it]

0                                                  None
1     [laptop : { battery : negative , keyboard : ne...
2     [ Laptop : { battery : negative , keyboard : n...
4     [ Laptop : { battery : ... , keyboard : ... , ...
6     [smartphone : { display : positive , camera : ...
7     [Laptop : { battery : negative , keyboard : ne...
9     [laptop : { battery : negative , keyboard : po...
10                                                 None
11                                                 None
12    [laptop : {battery : negative , keyboard : neg...
15    [ Laptop : { battery : negative , keyboard : n...
17    [laptop : { battery : ... , keyboard : ... , d...
18                                                 None
20    [Laptop : { battery : negative , keyboard : ne...
21    [ Laptop : { battery : negative , keyboard : n...
25                                                 None
26                                                 None
28    [Smartphone : { display : Positive , camer

In [ ]:
sample_reviews_examples_df.head()

,customer_id,customer_name,product_name,product_type,review_date,rating,review_sentiment,review_text,aspects_review,response,summary
0,CID041,Aisha Patel,Dell Inspiron 15,Laptop,01/01/2024,5,Positive,I bought this laptop for my son who is studyin...,"{ battery : Positive , keyboard : Positive, di...",It's fantastic to hear that the laptop you pur...,"The user purchased a laptop for their son, who..."
1,CID011,Liam Thomson,JBL Tune 500BT,Headphones,01/05/2024,1,Negative,I was very disappointed with these headphones....,"{ sound : Negative , comfort : Negative , char...",I'm truly sorry to hear about your disappointi...,The user expressed disappointment with poor so...
2,CID034,Maria García,Mi Power Bank 3i,Power Bank,01/10/2023,4,Positive,"Awesome power bank, it charges my phone very f...","{ battery : Positive , charging : Positive }",Thank you for your positive review of our powe...,The user praises the power bank for its fast c...
4,CID051,Emily Nguyen,JBL Tune 500BT,Headphones,05/25/2023,5,Positive,"I love these headphones, they are amazing. The...","{ sound : Positive , comfort : Positive , char...",Thank you for your wonderful feedback on our h...,The user praises the headphones for their exce...
6,CID043,Sofia Chen,Dell Inspiron 15,Laptop,05/20/2023,1,Negative,"I bought this laptop a month ago, and it's alr...","{ battery : Negative , keyboard : Negative , d...",I'm sorry to hear about your laptop issues and...,"The customer reports issues with their laptop,..."


In [ ]:
accuracy_list

[11.11111111111111, 5.555555555555555]

In [ ]:
# Calculate mean accuracy - sum of values in "accuracy list" divided by total values in the "accuracy_list"
mean_accuracy = sum(accuracy_list) / len(accuracy_list)

In [ ]:
mean_accuracy

8.333333333333332

**Evaluate the results**

In [ ]:
gold_examples

'[{"review_text":"This is a very bad power bank, it does not charge my phone properly. It takes a very long time to charge the power bank itself, and it drains very fast. It also does not charge my phone fully, it stops at around 80%. It also heats up very much, and sometimes it sparks and smokes. I think it is very dangerous and defective. I tried to return it, but the seller did not accept it. I feel cheated and scammed.","product_type":"Power Bank","aspects_review":"{ battery : Negative , charging : Negative }"},{"review_text":"This is a great laptop, I am very happy with it. Great battery life, it lasts for about 8 hours. It has great performance, it can handle multiple tasks and applications. Good storage capacity, it can store a lot of files and data. The laptop also has a great screen, it has a good resolution and viewing angle. It also has a great design, it\'s sturdy and durable and the keyboard\'s keys are good and strong. Overall I found it perfect, with basically no flaws a

In [ ]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for movie reviews

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths = [], []

    for example in json.loads(gold_examples):
        gold_input = example['review_text']
        user_input = [
            {
               user_message_template.format(review=gold_input)
            }
        ]

        try:
            prediction = generate_llama_response( few_shot_system_message , few_shot_examples_str  , user_input , 0.1 )

            print("prediction : " + prediction + "\n")

            prediction_extracted = extract_sentiment(prediction)
            predicted_product_type = extract_product_type(prediction_extracted)
            predicted_aspect_based_sentiment = extract_aspect_based_sentiment(prediction_extracted)

            final_prediction = predicted_product_type + ":" + predicted_aspect_based_sentiment

            # sentiment = match.group().lower() if match else "Sentiment not found."
            print("model_prediction : " + final_prediction.lower() + "\n")

            model_predictions.append(final_prediction.lower()) # <- removes extraneous white space and lowercases output
            ground_truths.append( example['product_type'].lower() + ":" + example['aspects_review'].lower() )

            print("ground truth : " + example['product_type'].lower() + ":" + example['aspects_review'].lower() + "\n")

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    return micro_f1_score

In [ ]:
# Evaluate the results
evaluate_prompt(few_shot_system_message,  gold_examples, user_message_template)

[INST]
<<SYS>>Perform aspect based sentiment analysis on customer reviews presented in the input delimited by triple backticks, that is, ```.

For each review presented as input:
- You need to identify firstly the product_type based on the review . Identify only 1 out of the 4 product_types : Laptop , Headphones , Power Bank , Smartphone
- After identifying the product_type , you will need to assign a sentiment polarity for each aspect of the product , it should only be 'Positive' or 'Negative' , nothing else is acceptable .
- Based on the product_type identified , give output only in 1 of the below 4 format :
1. if Laptop identified then - [ Laptop : { battery : ... , keyboard : ... , display : ... } ]
2. if Headphone identified then - [ Headphone : { sound : ... , comfort : ...  , charging : ... } ]
3. if Power Bank identified then - [ Power Bank : { battery : ...  , charging : ...  } ]
4. if Smartphone identified then - [ Smartphone : { display : ... , camera : ... , Battery : ... }

Llama.generate: prefix-match hit


prediction :   Sure, I can help you with that! Here are the output for each review in the format you specified:

1. If Laptop identified then - [ Laptop : { battery : negative , keyboard : negative } ]
2. If Headphone identified then - [ Headphone : { sound : negative , comfort : negative } ]
3. If Power Bank identified then - [ Power Bank : { battery : negative , charging : negative } ]
4. If Smartphone identified then - [ Smartphone : { display : positive , camera : positive } ]

Here are the reviews with my analysis:

1. [Smartphone]
User said: "I regret buying this phone."
Aspects: display, camera
Sentiment: negative

2. [Laptop]
User said: "Within a month, the laptop has become slow and laggy..."
Aspects: battery, keyboard
Sentiment: negative

3. [Power Bank]
User said: "This is a very poor power bank... does not work well."
Aspects: charging, capacity
Sentiment: negative

4. [Headphone]


model_prediction : laptop:{ battery : negative , keyboard : negative }

ground truth : power

Llama.generate: prefix-match hit


prediction :   Sure! Here is the output for each review in the format you requested:

1. If Laptop identified then - [Laptop : { battery : Positive , keyboard : Positive , display : Positive } ]
2. If Headphone identified then - [Headphone : { sound : Negative , comfort : Negative , charging : Negative } ]
3. If Power Bank identified then - [Power Bank : { battery : Negative , charging : Negative } ]
4. If Smartphone identified then - [Smartphone : { display : Positive , camera : Positive , Battery : Negative } ]

Here are the reviews with the product type and sentiment polarity for each aspect:

1. Laptop Review:
	* Product Type: Laptop
	* Aspects:
		+ Battery: Positive
		+ Keyboard: Positive
		+ Display: Positive
2. Headphone Review:
	* Product Type: Headphones
	* Aspects:
		+ Sound: Negative
		+ Comfort: Negative
		+ Charging: Neg

model_prediction : laptop:{ battery : positive , keyboard : positive , display : positive }

ground truth : laptop:{ battery : positive , keyboard : posi

...

Llama.generate: prefix-match hit


prediction :   Sure! Here's the output for each review in the format you specified:

1. [ Headphone : { sound : negative , comfort : negative } ]
2. [ Laptop : { battery : negative , keyboard : negative , display : positive } ]
3. [ Power Bank : { battery : negative , charging : negative } ]
4. [ Smartphone : { display : positive , camera : positive , battery : negative } ]

For the first review, the product type is Headphone, and the sentiment polarity for each aspect is:

* Sound: Negative
* Comfort: Negative

For the second review, the product type is Laptop, and the sentiment polarity for each aspect is:

* Battery: Negative
* Keyboard: Negative
* Display: Positive

For the third review, the product type is Power Bank, and the sentiment polarity for each aspect is:

* Battery: Negative
* Charging: Negative

For the fourth review, the product type is Smartphone, and the sentiment polarity for each aspect is:

* Display: Positive
* Camera

model_prediction : headphone:{ sound : negat

Llama.generate: prefix-match hit


prediction :   Sure! Here's the output for each review in the format you specified:

1. If Laptop identified then - [ Laptop : { battery : negative , keyboard : negative } ]
2. If Headphone identified then - [ Headphone : { sound : positive , comfort : negative } ]
3. If Power Bank identified then - [ Power Bank : { battery : positive , charging : positive } ]
4. If Smartphone identified then - [ Smartphone : { display : positive , camera : positive } ]

Here are the reviews with the corresponding product type and sentiment polarity for each aspect:

1. Review 1 (Smartphone) - [ Smartphone : { display : negative , camera : positive } ]
	* Aspects: Display, Camera
	* Sentiment Polarity: Negative, Positive
2. Review 2 (Laptop) - [ Laptop : { battery : negative , keyboard : negative } ]
	* Aspects: Battery, Keyboard
	* Sentiment Polarity: Negative, Negative
3. Review 3 (Power Bank) - [ Power Bank : { battery : positive

model_prediction : laptop:{ battery : negative , keyboard : negative 

0.375

## **Part 2: Parameter Efficient Fine-tuning:**


In [ ]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps xformers==0.0.27.post2 "trl<0.9.0" peft==0.12.0 accelerate==0.32.1 bitsandbytes==0.43.2
!pip install -q datasets==2.16.1 evaluate==0.4.1 rouge_score==0.1.2 bert_score==0.3.12
!pip install triton==3.0.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, bu

### **Import Necessary Libraries**


In [ ]:
# import the "torch" library
import torch

# import FastLanguageModel from unsloth library
from unsloth import FastLanguageModel

# import TrainingArguments from transformers
from transformers import TrainingArguments, EarlyStoppingCallback

# import evaluate library
import evaluate

# import locale library
import locale

# import SFTTrainer from trl
from trl import SFTTrainer
import numpy as np
import pandas as pd

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### **Dataset Preprocessing for Text Summarization**

Read the Dataset

Split the Datset

**Load Dataset**

In [ ]:
sample_reviews_df = pd.read_csv("/content/customer_reviews_dataset.csv")

In [ ]:
sample_reviews_df['dialogue'] = 'customer : ' + sample_reviews_df['review_text'] + '\n' + 'response : ' + sample_reviews_df['response'] + '\n'

In [ ]:
sample_reviews_df['id'] = sample_reviews_df['customer_id']

In [ ]:
sample_reviews_df = sample_reviews_df[['id', 'review_sentiment' ,'dialogue','summary']]

In [ ]:
# Checking the count of positive and negative values in the dataframe
sample_reviews_df.review_sentiment.value_counts()

,count
review_sentiment,
Positive,15
Negative,15


In [ ]:
sample_reviews_df.head()

,id,review_sentiment,dialogue,summary
0,CID041,Positive,customer : I bought this laptop for my son who...,"The user purchased a laptop for their son, who..."
1,CID011,Negative,customer : I was very disappointed with these ...,The user expressed disappointment with poor so...
2,CID034,Positive,"customer : Awesome power bank, it charges my p...",The user praises the power bank for its fast c...
3,CID032,Negative,customer : I bought this phone mainly for its ...,The user expressed disappointment with the pho...
4,CID051,Positive,"customer : I love these headphones, they are a...",The user praises the headphones for their exce...


**Split Dataset**

In [ ]:
# Separate positive and negative reviews
positive_reviews = sample_reviews_df[sample_reviews_df['review_sentiment'] == 'Positive']
negative_reviews = sample_reviews_df[sample_reviews_df['review_sentiment'] == 'Negative']

# Sample 2 positive and 2 negative reviews for gold examples
positive_gold_examples = positive_reviews.sample(2, random_state=40)
negative_gold_examples = negative_reviews.sample(2, random_state=40)

# Concatenate positive and negative gold examples
sample_reviews_gold_examples_df = pd.concat([positive_gold_examples, negative_gold_examples])

# Create the training set by excluding gold examples
sample_reviews_examples_df = sample_reviews_df.drop(index=sample_reviews_gold_examples_df.index)

# Print the shapes of the datasets
print("Training Set Shape:", sample_reviews_examples_df.shape)
print("Gold Examples Shape:", sample_reviews_gold_examples_df.shape)


Training Set Shape: (26, 4)
Gold Examples Shape: (4, 4)


In [ ]:
sample_reviews_gold_examples_df.head()

,id,review_sentiment,dialogue,summary
0,CID041,Positive,customer : I bought this laptop for my son who...,"The user purchased a laptop for their son, who..."
26,CID067,Positive,customer : The laptop is good enough for the p...,"The customer praises the laptop's performance,..."
1,CID011,Negative,customer : I was very disappointed with these ...,The user expressed disappointment with poor so...
22,CID076,Negative,"customer : This is a very poor power bank, it ...",The customer is dissatisfied with their power ...


### **Model Setup for Fine-tuning**

(A) Load Model Name (1 Marks)

(B) Create Examples (2 Marks)

(C) Initialize Model (2 Marks)

**Load llama-2-7b-bnb-4bit model Name**

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-2-7b-bnb-4bit",
    # model_name="meta-llama/Llama-2-7b-chat-hf"
    max_seq_length=2048,
    dtype=None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit=True # Use 4bit quantization to reduce memory usage.
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-0

In [ ]:
tokenizer

LlamaTokenizerFast(name_or_path='unsloth/llama-2-7b-bnb-4bit', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
EOS_TOKEN = tokenizer.eos_token

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0, # Supports any, but = 0 is optimized
    bias="none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
def create_examples_with_seed(dataset, n=2, random_seed=None):
    """
    Return two DataFrames with randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (text + label)
        n (int): number of examples of each class to be selected
        random_seed (int): seed for reproducibility (default is None)

    Output:
        few_shot_examples_df (DataFrame): A DataFrame with examples in random order
        new_df (DataFrame): A new DataFrame excluding selected examples
    """

    positive_reviews = (dataset.review_sentiment == 'Positive')
    negative_reviews = (dataset.review_sentiment == 'Negative')
    columns_to_select = ['id', 'review_sentiment' ,'dialogue','summary']

    # Set a fixed random seed for reproducibility
    np.random.seed(random_seed)

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    few_shot_examples_df = pd.concat([positive_examples, negative_examples])
    # sampling without replacement is equivalent to random shuffling
    few_shot_examples_df = few_shot_examples_df.sample(2 * n, replace=False)

    # Create a new DataFrame excluding selected examples
    new_df = dataset.drop(index=few_shot_examples_df.index)

    return few_shot_examples_df, new_df


In [ ]:
few_shot_examples_df , sample_reviews_train_examples_df = create_examples_with_seed(sample_reviews_examples_df, n=2 , random_seed = None)

**Creating examples using seed value used earlier and n=2**

In [ ]:
sample_reviews_train_examples_df , sample_reviews_validation_examples_df = create_examples_with_seed(sample_reviews_examples_df, n=2 , random_seed=None)

In [ ]:
sample_reviews_train_examples_df

,id,review_sentiment,dialogue,summary
8,CID029,Negative,customer : I bought these headphones a week ag...,The customer purchased poor-quality headphones...
18,CID078,Negative,customer : I bought this laptop quite recently...,The customer is unhappy with their recently pu...
11,CID040,Positive,customer : Recently tried a power bank sharing...,The customer reviews a power bank sharing serv...
23,CID075,Positive,customer : Great for listening to music or pod...,Customer praises headphones for good sound qua...


In [ ]:
training_dataset = datasets.Dataset.from_pandas(sample_reviews_train_examples_df)
validation_dataset = datasets.Dataset.from_pandas(sample_reviews_validation_examples_df)


In [ ]:
def prompt_formatter(example, prompt_template):
    instruction='Summarize the following dialogue'
    dialogue=example["dialogue"]
    summary=example["summary"]

    formatted_prompt = prompt_template.format(instruction, dialogue, summary)

    return {'formatted_prompt': formatted_prompt}

In [ ]:
formatted_training_dataset = training_dataset.map(
    prompt_formatter,
    fn_kwargs={'prompt_template': alpaca_prompt}
)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
formatted_validation_dataset = validation_dataset.map(
    prompt_formatter,
    fn_kwargs={'prompt_template': alpaca_prompt}
)

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

**Initialize Model Parameters**

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    # Input formatted training dataset
    train_dataset=formatted_training_dataset,
    # Input formatted validation dataset
    eval_dataset=formatted_validation_dataset,
    dataset_text_field = "formatted_prompt",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False, # Increases efficiency for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=20,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs"
    )
)

Map (num_proc=2):   0%|          | 0/4 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/22 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


#### *Start of Training*

### **Train and Save the Model**

Train the Model

Save the Model

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
3.826 GB of memory reserved.


**Train Model**

In [ ]:
# Start Training
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 20
 "-____-"     Number of trainable parameters = 39,976,960
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
1,0.897700
2,0.897700
3,0.886900
4,0.863300
5,0.821600
6,0.764200
7,0.690400
8,0.617200
9,0.553800
10,0.496900


In [ ]:
trainer_stats

TrainOutput(global_step=20, training_loss=0.523023248463869, metrics={'train_runtime': 99.3786, 'train_samples_per_second': 1.61, 'train_steps_per_second': 0.201, 'total_flos': 1127119676866560.0, 'train_loss': 0.523023248463869, 'epoch': 20.0})

#### *Inference*

In [ ]:
test_dataset = datasets.Dataset.from_pandas(sample_reviews_gold_examples_df)

In [ ]:
test_dataset[0]

{'id': 'CID041',
 'review_sentiment': 'Positive',
 'dialogue': "customer : I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design. The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.\nresponse : It's fantastic to hear that the laptop you purchased for your son has met his needs and expectations, especially in his engineering studies. A good battery life, fast performance, and sleek design are essential for a student's productivity. The comfortable keyboard and bright screen further enhance the usability of the laptop. If you ever encounter any issues or have questions about the laptop, please feel free to reach out for support. We're here to ensure that your experience continues to be positive. Thank you for choosing our product and taking the time to share your satisfaction!\n",

In [ ]:
instruction = "Summarize the following dialogue"
test_dialogue = test_dataset[0]['dialogue']
test_summary = test_dataset[0]['summary']

In [ ]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
       

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction,
        test_dialogue,
        "", # leave output blank for generation
    )
], return_tensors="pt").to("cuda")

In [ ]:
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    use_cache=True,
    do_sample=True,
    temperature=0.2
)

In [ ]:
print(tokenizer.batch_decode(outputs)[0])

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following dialogue

### Input:
customer : I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design. The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.
response : It's fantastic to hear that the laptop you purchased for your son has met his needs and expectations, especially in his engineering studies. A good battery life, fast performance, and sleek design are essential for a student's productivity. The comfortable keyboard and bright screen further enhance the usability of the laptop. If you ever encounter any issues or have questions about the laptop, please feel free to reach out for sup

In [ ]:
test_summary

'The user purchased a laptop for their son, who is studying engineering. They are satisfied with its battery life, fast performance, sleek design, comfortable keyboard, and bright screen, and its one-year warranty.'

The following code allows us to run shell commands on Colab (we need shell commands to check file sizes of the estimated models).

In [ ]:
import locale

In [ ]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [ ]:
lora_model_name = "dialogue-summarizer-llama"

**Save Model**

In [ ]:
# save the model using save_pretrained function from model
model.save_pretrained(lora_model_name)

In [ ]:
!ls -lh {lora_model_name}

total 153M
-rw-r--r-- 1 root root  727 Sep  5 21:26 adapter_config.json
-rw-r--r-- 1 root root 153M Sep  5 21:26 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Sep  5 21:26 README.md


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r {lora_model_name} /content/drive/MyDrive/

### **Evaluate Model Performance**

(A) Load Base Model (2 Marks)

(B) Evaluate Performance of Base Model (3 Marks)

(C) Load Trained Model (2 Marks)

(D) Evaluate Performance of Trained Model (3 Marks)

#### *Llama 2 Base Model Performance*

In [ ]:
import torch
torch.cuda.empty_cache()

**Load Base Model**

In [ ]:
baseline_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-2-7b-bnb-4bit",
    max_seq_length=2048,
    dtype=None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit=True, # Use 4bit quantization to reduce memory usage.
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
FastLanguageModel.for_inference(baseline_model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-0

In [ ]:
test_dataset = datasets.Dataset.from_pandas(sample_reviews_gold_examples_df)

#### *Single Inference*

In [ ]:
# This helps to free the RAM
torch.cuda.empty_cache()

In [ ]:
instruction = "Summarize the following dialogue"
test_dialogue = test_dataset[0]['dialogue']
test_summary = test_dataset[0]['summary']

In [ ]:
input = tokenizer(
    alpaca_prompt.format(
        instruction,
        test_dialogue,
        ""
    ), return_tensors="pt"
).to("cuda")

In [ ]:
output = baseline_model.generate(
    **input,
    max_new_tokens=128,
    use_cache=True,
    do_sample=True,
    temperature=0.2
)

In [ ]:
print(tokenizer.decode(output[0]))

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following dialogue

### Input:
customer : I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design. The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.
response : It's fantastic to hear that the laptop you purchased for your son has met his needs and expectations, especially in his engineering studies. A good battery life, fast performance, and sleek design are essential for a student's productivity. The comfortable keyboard and bright screen further enhance the usability of the laptop. If you ever encounter any issues or have questions about the laptop, please feel free to reach out for sup

#### *Batch Inference*

In [ ]:
torch.cuda.empty_cache()

In [ ]:
instruction = "Summarize the following dialogue"

In [ ]:
test_dialogues = [sample['dialogue'] for sample in test_dataset]
test_summaries = [sample['summary'] for sample in test_dataset]

In [ ]:
def extract_summary_from_string(input_string):
    try:
        # Assuming the response is between ### Response: and </s>
        summary_start = input_string.rfind('### Response:\n') + 14 # number of characters in '### Response:\n'
        summary_end = input_string.rfind('</s>')
        summary_str = input_string[summary_start:summary_end]

        return summary_str
    except Exception as e:
        print(f"Error decoding string: {e}")
        return None

In [ ]:
predicted_summaries = []

In [ ]:
for sample_dialogue in test_dialogues:
  input = tokenizer(
    alpaca_prompt.format(
        instruction,
        sample_dialogue,
        ""
    ), return_tensors="pt"
  ).to("cuda")

  outputs = baseline_model.generate(
    **input,
    max_new_tokens=256,
    use_cache=True
  )

  predicted_summary = tokenizer.decode(outputs[0])

  output_str = extract_summary_from_string(predicted_summary)

  predicted_summaries.append(output_str)

#### *Evaluate*

**Evaluate Performance of Base Model**

In [ ]:
bert_scorer = evaluate.load("bertscore")

In [ ]:
# Provide Prediction Summaries and Test Summaries as input
score = bert_scorer.compute(
    predictions=predicted_summaries,
    references=test_summaries,
    lang="en",
    rescale_with_baseline=True
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(score)

{'precision': [0.05797911062836647, 0.266434907913208, -0.08124058693647385, 0.10214785486459732], 'recall': [0.5340350270271301, 0.45830297470092773, -0.0826530009508133, 0.47137415409088135], 'f1': [0.28634724020957947, 0.3616742193698883, -0.0801893100142479, 0.28141146898269653], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.44.2)-rescaled'}


In [ ]:
# The given dictionary with precision, recall, and f1 values
#  Calculate Average Bert Score . Average Bert Score is sum of f1 score divided by number of samples
bert_score = {
    'precision': [0.05797911062836647, 0.266434907913208, -0.08124058693647385, 0.10214785486459732],
    'recall': [0.5340350270271301, 0.45830297470092773, -0.0826530009508133, 0.47137415409088135],
    'f1': [0.28634724020957947, 0.3616742193698883, -0.0801893100142479, 0.28141146898269653],
    'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.44.2)-rescaled'
}

# Extract the F1 values from the dictionary
f1_values = bert_score['f1']

# Calculate the average of the F1 values
average_f1_score = sum(f1_values) / len(f1_values)

# Print the result
print(f"Average BERT Score (F1): {average_f1_score}")


Average BERT Score (F1): 0.2123109046369791


#### *Llama 2 Trained (Fine-tuned) Model Summarizer*

In [ ]:
lora_model_name = "dialogue-summarizer-llama"

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [ ]:
!cp -r /content/drive/MyDrive/{lora_model_name} .

**2.5 (C) Load Trained Model**

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=lora_model_name,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
       

In [ ]:
test_dataset = datasets.Dataset.from_pandas(sample_reviews_gold_examples_df)

#### *Single Inference*

In [ ]:
torch.cuda.empty_cache()

In [ ]:
instruction = "Summarize the following dialogue"
test_dialogue = test_dataset[0]['dialogue']
test_summary = test_dataset[0]['summary']

In [ ]:
input = tokenizer(
    alpaca_prompt.format(
        instruction,
        test_dialogue,
        ""
    ), return_tensors="pt"
).to("cuda")

In [ ]:
output = model.generate(
    **input,
    max_new_tokens=128,
    use_cache=True,
    do_sample=True,
    temperature=0.2
)

In [ ]:
print(tokenizer.decode(output[0]))

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following dialogue

### Input:
customer : I bought this laptop for my son who is studying engineering. He is very happy with it. It has a good battery life, fast performance, and a sleek design. The keyboard is comfortable and the screen is bright. The laptop came with a one-year warranty and a free antivirus software. I think it is a great value for money.
response : It's fantastic to hear that the laptop you purchased for your son has met his needs and expectations, especially in his engineering studies. A good battery life, fast performance, and sleek design are essential for a student's productivity. The comfortable keyboard and bright screen further enhance the usability of the laptop. If you ever encounter any issues or have questions about the laptop, please feel free to reach out for sup

#### *Batch Inference*

In [ ]:
torch.cuda.empty_cache()

In [ ]:
instruction = "Summarize the following dialogue"

In [ ]:
# test_size = 4

In [ ]:
test_dialogues = [sample['dialogue'] for sample in test_dataset]
test_summaries = [sample['summary'] for sample in test_dataset]

In [ ]:
def extract_summary_from_string(input_string):
    try:
        # Assuming the response is between ### Response: and </s>
        summary_start = input_string.rfind('### Response:\n') + 14 # number of characters in '### Response:\n'
        summary_end = input_string.rfind('</s>')
        summary_str = input_string[summary_start:summary_end]

        return summary_str
    except Exception as e:
        print(f"Error decoding string: {e}")
        return None

In [ ]:
predicted_summaries = []

In [ ]:
for sample_dialogue in test_dialogues:
  input = tokenizer(
    alpaca_prompt.format(
        instruction,
        sample_dialogue,
        ""
    ), return_tensors="pt"
  ).to("cuda")

  outputs = model.generate(
    **input,
    max_new_tokens=256,
    use_cache=True
  )

  predicted_summary = tokenizer.decode(outputs[0])

  output_str = extract_summary_from_string(predicted_summary)

  predicted_summaries.append(output_str)

#### *Evaluate*

**Evaluate Performance of Trained Model**

In [ ]:
# Input prediction summaries and test summaries in bert scorer
score = bert_scorer.compute(
    predictions= predicted_summaries,
    references=test_summaries,
    lang="en",
    rescale_with_baseline=True
)

In [ ]:
# Print Bert score
print(score)

{'precision': [0.45185530185699463, 0.42932572960853577, 0.6199865341186523, 0.5423881411552429], 'recall': [0.5248703360557556, 0.5800672769546509, 0.4360915720462799, 0.5239931344985962], 'f1': [0.48895028233528137, 0.5044614672660828, 0.5272566676139832, 0.5339332222938538], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.44.2)-rescaled'}


In [ ]:
# The given dictionary with precision, recall, and f1 values
#  Calculate Average Bert Score . Average Bert Score is sum of f1 score divided by number of samples
bert_score = {
    'precision': [0.45185530185699463, 0.42932572960853577, 0.6199865341186523, 0.5423881411552429],
    'recall': [0.5248703360557556, 0.5800672769546509, 0.4360915720462799, 0.5239931344985962],
    'f1': [0.48895028233528137, 0.5044614672660828, 0.5272566676139832, 0.5339332222938538],
    'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.44.2)-rescaled'
}

# Extract the F1 values from the dictionary
f1_values = bert_score['f1']

# Calculate the average of the F1 values
average_f1_score = sum(f1_values) / len(f1_values)

# Print the result
print(f"Average BERT Score (F1): {average_f1_score}")


Average BERT Score (F1): 0.5136504098773003


## **Part 3. Observations, Insights and Business Recommendations**

**Part 1**

* The base model was implemented for sentiment classification, the model perfomed poorly with 5% accuracy, hence generating inaccurate product aspect_review sentiments.

* The model would require additional fine-tuning to achieve a better model prediction. For very specific task like aspect based review, a fine-tuned model might be better off using compared to a general base model.

* Few gold examples have been carefully curated to represent accurate data for model evaluation purposes. It is expected the model  will perform well with just a few labeled high quality gold samples, which suggests that the model can handle real-world scenarios where labeled data is scarce. However, given the base model in this case the model performed poorly with 5% accuracy.




**Part 2**

* Model fine-tuning was implemented using performance efficient fune-tuning method to enhance the capability of the LLM and evaluate the quality of summaries.

* The llama based model training loss redcued significantly from 0.8 - 0.2%, however the F1-Score showed a low score of 0.2%, indicated the model requires further fine-tuning.

* Fine-tuning the base model for summarization using PEFT method improved the F1 score to 0.5%.

* Bertscore evalaution metric was used in other to capture semeantic difference between the dialogue and summaries.




